In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import requests
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix
import re
from tqdm import tqdm
import time
df = pd.read_csv('Data/bbc_news_reasons.csv')
# Drop all rows that contain any missing values
df = df.dropna()
df.reset_index

# Check the DataFrame after dropping rows with missing values
df

,text_id,text,original_code,replicated_code,model_code,reason
0,0,Bath faced with Tindall ultimatum\n\nMike Tind...,sport,sport,sport,"This news story is about a rugby player, Mike ..."
1,1,Big guns ease through in San Jose\n\nTop-seede...,sport,sport,sport,"The text is reporting on a tennis tournament, ..."
2,2,Chinese dam firm 'defies Beijing'\n\nThe China...,business,business,politics,This news story is about a Chinese company def...
3,3,Wenger offers Mutu hope\n\nArsenal boss Arsene...,sport,sport,sport,This news story is about a football player who...
4,4,Gerrard happy at Anfield\n\nLiverpool captain ...,sport,sport,sport,The article is about Liverpool captain Steven ...
...,...,...,...,...,...,...
95,95,Who do you think you are?\n\nThe real danger i...,tech,tech,tech,The article discusses the dangers of data thef...
96,96,Koubek suspended after drugs test\n\nStefan Ko...,sport,sport,sport,"The news story is about a tennis player, Stefa..."
97,97,China 'blocks Google news site'\n\nChina has b...,tech,politics,politics,This news story is about China's censorship of...
98,98,Leeds v Saracens (Fri)\n\nHeadingley\n\nFriday...,sport,sport,sport,The text is reporting on a rugby match between...


In [5]:
import pandas as pd
import ollama
import re
import time
from tqdm import tqdm

# Example codebook
CODEBOOK = """
business: Tweets or news articles related to companies, financial markets, economic issues, or industry trends.
entertainment: Tweets or news articles related to movies, music, celebrities, cultural events, or leisure activities.
politics: Tweets or news articles related to government, elections, political parties, policies, or international relations.
sport: Tweets or news articles related to sporting events, athletes, competitions, or athletic activities.
tech: Tweets or news articles related to technology advancements, innovation, science, or tech companies.
"""

def prepare_prompt(text):
    system_prompt = f"""
    You are a qualitative coder who is annotating BBC news stories.
    Your task is to assign ONE code from the codebook and explain your reasoning.
    
    YOU MUST INCLUDE BOTH A CODE AND A REASON IN YOUR RESPONSE.
    
    REQUIRED FORMAT:
    Code: [single word category]
    Reason: [at least one sentence explaining why you chose this code]
    
    Codebook:
    ---
    {CODEBOOK}
    ---
    Text:
    ---
    {text}
    ---
    """
    return system_prompt

def classify_text(text, retry_count=3, retry_delay=1):
    """Classify text using local Ollama instance with retry logic."""
    for attempt in range(retry_count):
        try:
            # Generate response using ollama
            response = ollama.generate(
                model='llama3.2:latest',  # Updated to match your installed model
                prompt=prepare_prompt(text),
                stream=False
            )
            
            output = response['response']
            
            if not output:
                raise ValueError("Empty response from model")
            
            print("\nRaw model output:", output)  # Debug output
                
            # Extract code and reasoning using patterns
            code_patterns = [
                r'Code:\s*([a-zA-Z]+)(?=\s*Reason:)',  # Primary pattern
                r'Code:\s*([a-zA-Z]+)',  # Backup pattern
                r'\b(sport|business|politics|entertainment|tech)\b'  # Last resort
            ]
            
            code = "Unknown"
            for pattern in code_patterns:
                match = re.search(pattern, output, re.IGNORECASE)
                if match:
                    code = match.group(1).lower().strip()
                    break
            
            reasoning_patterns = [
                r'Reason:\s*([\s\S]+?)(?=\s*(?:Code:|$))',
                r'Reason:\s*([\s\S]+)$'
            ]
            
            reasoning = "No reasoning provided"
            for pattern in reasoning_patterns:
                match = re.search(pattern, output, re.IGNORECASE)
                if match:
                    reasoning = match.group(1).strip()
                    break
            
            if reasoning == "No reasoning provided" and output:
                reasoning = output.strip()
            
            # Force code to 'business' if text contains specific economic keywords
            economic_keywords = ['economy', 'recession', 'economic', 'gdp', 'market']
            if code == "Unknown" and any(keyword in text.lower() for keyword in economic_keywords):
                code = "business"
            
            return code, reasoning
                
        except Exception as e:
            if attempt == retry_count - 1:
                return "Error", f"Processing failed after {retry_count} attempts: {str(e)}"
            time.sleep(retry_delay * (attempt + 1))
            continue

def process_dataframe(df, batch_size=50, save_interval=200):
    """Process the dataframe with progress tracking and periodic saving."""
    df_processed = df.copy()
    
    # Initialize columns for results
    if 'model_prediction' not in df_processed.columns:
        df_processed['model_prediction'] = None
    if 'reasoning' not in df_processed.columns:
        df_processed['reasoning'] = None
    
    unprocessed_mask = df_processed['model_prediction'].isna()
    total_unprocessed = unprocessed_mask.sum()
    
    if total_unprocessed == 0:
        print("All rows have already been processed!")
        return df_processed
    
    print(f"Processing {total_unprocessed} unprocessed rows...")
    
    with tqdm(total=total_unprocessed) as pbar:
        processed_count = 0
        
        for idx in df_processed[unprocessed_mask].index:
            try:
                text = df_processed.loc[idx, 'text']
                code, reasoning = classify_text(text)
                
                df_processed.loc[idx, 'model_prediction'] = code
                df_processed.loc[idx, 'reasoning'] = reasoning
                
                processed_count += 1
                pbar.update(1)
                
                # Save intermediate results
                if processed_count % save_interval == 0:
                    print(f"\nSaving intermediate results after processing {processed_count} rows...")
                    df_processed.to_csv('ollama_intermediate_results.csv', index=False)
                
                # Add delay to prevent overwhelming the model
                if processed_count % batch_size == 0:
                    time.sleep(1)
                    
            except Exception as e:
                print(f"\nError processing row {idx}: {str(e)}")
                continue
    
    # Print summary
    success_count = df_processed['model_prediction'].notna().sum() - df_processed['model_prediction'].eq('Error').sum()
    error_count = df_processed['model_prediction'].eq('Error').sum()
    print(f"\nProcessing complete!")
    print(f"Successfully processed: {success_count} rows")
    print(f"Errors: {error_count} rows")
    
    return df_processed

# Example usage:
if __name__ == "__main__":
    # Load your DataFrame here
    # df = pd.read_csv('your_input_file.csv')
    df_results = process_dataframe(df)
    # df_results.to_csv('ollama_results.csv', index=False)
    print("Ready to process! Load your DataFrame and call process_dataframe()")

Processing 98 unprocessed rows...


  1%|          | 1/98 [00:10<16:19, 10.10s/it]


Raw model output: Code: politics
Reason: The text discusses a specific player (Mike Tindall) and his contract negotiations with a sports club (Bath), which is tied to the sport of rugby, but the main focus is on the agent's warning about potential defection due to the club's unwillingness to meet the player's demands. This issue is related to the broader topic of politics in the context of club management and ownership, making it a relevant category under "politics".


  2%|▏         | 2/98 [00:14<10:53,  6.81s/it]


Raw model output: Code: sports
Reason: The text primarily focuses on tennis events, including matches between players such as Andy Roddick, Andre Agassi, Jurgen Melzer, Xavier Malisse, Hyung-Taik Lee, Cyril Saulnier, and Tomas Zib, which suggests that it is about sporting activities.


...


Raw model output: Code: politics
Reason: I assigned the code "politics" because the majority of the text discusses government actions and policies related to censorship, internet control, and freedom of information, all of which fall under the scope of politics. The mention of a "net police force", government-approved content restrictions, and the involvement of Beijing in controlling Google News also indicate that the story revolves around political issues and power dynamics at play.


 99%|█████████▉| 97/98 [09:21<00:05,  5.43s/it]


Raw model output: Code: sport
Reason: This annotation is coded as "sport" because the text primarily discusses a specific sporting event, a rugby match between Leeds and Saracens at Headingley Stadium on February 25th, 2000. The details provided about the teams' lineups, player substitutions, and the game's progress indicate that the text is focused on reporting the sports news aspect of the event.


100%|██████████| 98/98 [09:28<00:00,  5.80s/it]


Raw model output: Code: politics
Reason: The text primarily focuses on government policies, international relations, and security measures, specifically the UN Convention on Seafarers' Identity Documents, its revised version, and the implementation of biometric technology for tracking seafarers. Additionally, the quote from Cynthia Musselman mentions "increasing security on the seas as well as border control and protection," which further emphasizes the political context of the story.

Processing complete!
Successfully processed: 98 rows
Errors: 0 rows
Ready to process! Load your DataFrame and call process_dataframe()


In [6]:
df_results

,text_id,text,original_code,replicated_code,model_code,reason,model_prediction,reasoning
0,0,Bath faced with Tindall ultimatum\n\nMike Tind...,sport,sport,sport,"This news story is about a rugby player, Mike ...",politics,The text discusses a specific player (Mike Tin...
1,1,Big guns ease through in San Jose\n\nTop-seede...,sport,sport,sport,"The text is reporting on a tennis tournament, ...",sports,"The text primarily focuses on tennis events, i..."
2,2,Chinese dam firm 'defies Beijing'\n\nThe China...,business,business,politics,This news story is about a Chinese company def...,politics,The text primarily focuses on government actio...
3,3,Wenger offers Mutu hope\n\nArsenal boss Arsene...,sport,sport,sport,This news story is about a football player who...,politics,The text discusses the actions of Chelsea FC i...
4,4,Gerrard happy at Anfield\n\nLiverpool captain ...,sport,sport,sport,The article is about Liverpool captain Steven ...,politics,The article discusses Liverpool captain Steven...
...,...,...,...,...,...,...,...,...
95,95,Who do you think you are?\n\nThe real danger i...,tech,tech,tech,The article discusses the dangers of data thef...,politics,The article discusses government regulation an...
96,96,Koubek suspended after drugs test\n\nStefan Ko...,sport,sport,sport,"The news story is about a tennis player, Stefa...",sports,The text primarily discusses Stefan Koubek's s...
97,97,China 'blocks Google news site'\n\nChina has b...,tech,politics,politics,This news story is about China's censorship of...,politics,"I assigned the code ""politics"" because the maj..."
98,98,Leeds v Saracens (Fri)\n\nHeadingley\n\nFriday...,sport,sport,sport,The text is reporting on a rugby match between...,sport,"This annotation is coded as ""sport"" because th..."


In [8]:
results = df_results

In [11]:
def calculate_similarity_metrics(results_df, ground_truth_col, prediction_col):
    """
    Calculate various similarity metrics between two columns
    """
    # Remove rows where either column has missing values
    valid_rows = results_df[[ground_truth_col, prediction_col]].dropna()
    
    if len(valid_rows) == 0:
        return {
            'accuracy': 0,
            'kappa': 0,
            'matching_cases': 0,
            'total_cases': 0,
            'percentage_match': 0
        }
    
    # Calculate metrics
    accuracy = accuracy_score(valid_rows[ground_truth_col], valid_rows[prediction_col])
    kappa = cohen_kappa_score(valid_rows[ground_truth_col], valid_rows[prediction_col])
    matching_cases = (valid_rows[ground_truth_col] == valid_rows[prediction_col]).sum()
    total_cases = len(valid_rows)
    percentage_match = (matching_cases / total_cases) * 100
    
    return {
        'accuracy': accuracy,
        'kappa': kappa,
        'matching_cases': matching_cases,
        'total_cases': total_cases,
        'percentage_match': percentage_match
    }

def analyze_code_similarities(results_df):
    """
    Analyze similarities between model outputs and human categorizations
    """
    # Comparisons to make
    comparisons = [
        ('original_code', 'model_code'),
        ('original_code', 'model_prediction'),
        ('replicated_code', 'model_code'),
        ('replicated_code', 'model_prediction')
    ]
    
    # Calculate metrics for each comparison
    results_dict = {}
    for truth_col, pred_col in comparisons:
        results_dict[f"{truth_col}_vs_{pred_col}"] = calculate_similarity_metrics(
            results_df, truth_col, pred_col
        )
    
    # Create a summary dataframe
    summary_df = pd.DataFrame({
        'Comparison': [k.replace('_', ' ').title() for k in results_dict.keys()],
        'Accuracy': [v['accuracy'] for v in results_dict.values()],
        'Kappa Score': [v['kappa'] for v in results_dict.values()],
        'Matching Cases': [v['matching_cases'] for v in results_dict.values()],
        'Total Cases': [v['total_cases'] for v in results_dict.values()],
        'Match Percentage': [v['percentage_match'] for v in results_dict.values()]
    })
    
    # Format the percentage column
    summary_df['Match Percentage'] = summary_df['Match Percentage'].round(2).astype(str) + '%'
    
    # Create detailed confusion matrices
    confusion_matrices = {}
    for truth_col, pred_col in comparisons:
        valid_rows = results_df[[truth_col, pred_col]].dropna()
        if len(valid_rows) > 0:
            matrix = confusion_matrix(
                valid_rows[truth_col], 
                valid_rows[pred_col],
                labels=sorted(results_df[truth_col].unique())
            )
            confusion_matrices[f"{truth_col}_vs_{pred_col}"] = matrix
    
    return summary_df, confusion_matrices

# Run the analysis with the results dataframe
summary_df, confusion_matrices = analyze_code_similarities(results)

# Display the summary
print("\nSimilarity Analysis Summary:")
print("=" * 100)
print(summary_df.to_string(index=False))
print("\nDetailed Analysis:")
print("=" * 100)

# Determine which model is more similar to human coding
original_code_comparison = summary_df[summary_df['Comparison'].str.contains('Original Code')]
replicated_code_comparison = summary_df[summary_df['Comparison'].str.contains('Replicated Code')]

print("\nComparison with Original Code:")
better_for_original = original_code_comparison.iloc[original_code_comparison['Accuracy'].argmax()]
print(f"Better model: {better_for_original['Comparison']}")
print(f"Accuracy: {better_for_original['Accuracy']:.2%}")
print(f"Kappa Score: {better_for_original['Kappa Score']:.3f}")

print("\nComparison with Replicated Code:")
better_for_replicated = replicated_code_comparison.iloc[replicated_code_comparison['Accuracy'].argmax()]
print(f"Better model: {better_for_replicated['Comparison']}")
print(f"Accuracy: {better_for_replicated['Accuracy']:.2%}")
print(f"Kappa Score: {better_for_replicated['Kappa Score']:.3f}")

# Print confusion matrices with labels
for name, matrix in confusion_matrices.items():
    print(f"\nConfusion Matrix for {name}:")
    categories = sorted(results[name.split('_vs_')[0]].unique())
    print("\nCategories:", categories)
    print(matrix)


Similarity Analysis Summary:
                         Comparison  Accuracy  Kappa Score  Matching Cases  Total Cases Match Percentage
        Original Code Vs Model Code  0.877551     0.842823              86           98           87.76%
  Original Code Vs Model Prediction  0.489796     0.358135              48           98           48.98%
      Replicated Code Vs Model Code  0.816327     0.761557              80           98           81.63%
Replicated Code Vs Model Prediction  0.489796     0.337659              48           98           48.98%

Detailed Analysis:

Comparison with Original Code:
Better model: Original Code Vs Model Code
Accuracy: 87.76%
Kappa Score: 0.843

Comparison with Replicated Code:
Better model: Replicated Code Vs Model Code
Accuracy: 81.63%
Kappa Score: 0.762

Confusion Matrix for original_code_vs_model_code:

Categories: ['business', 'entertainment', 'politics', 'sport', 'tech']
[[15  0  5  0  0]
 [ 2 14  1  0  0]
 [ 0  0 25  0  0]
 [ 0  0  2 23  0]
 [ 1  

In [13]:
import pandas as pd
import ollama
import re
import time
from tqdm import tqdm

# Example codebook
CODEBOOK = """
business: Tweets or news articles related to companies, financial markets, economic issues, or industry trends.
entertainment: Tweets or news articles related to movies, music, celebrities, cultural events, or leisure activities.
politics: Tweets or news articles related to government, elections, political parties, policies, or international relations.
sport: Tweets or news articles related to sporting events, athletes, competitions, or athletic activities.
tech: Tweets or news articles related to technology advancements, innovation, science, or tech companies.
"""

# Few-shot examples to help guide the model
EXAMPLES = """
Example 1:
Text: "Manchester United secure dramatic win over Liverpool with last-minute goal from Rashford"
Code: sport
Reason: This is clearly a sports article about a football match between Manchester United and Liverpool, featuring a specific player and match outcome.

Example 2:
Text: "Apple unveils new iPhone with revolutionary AI capabilities"
Code: tech
Reason: The article is about a technology product launch and discusses technological innovations.

Example 3:
Text: "Global stock markets tumble amid inflation fears"
Code: business
Reason: This covers financial markets and economic issues, specifically focusing on stock market performance.

Example 4:
Text: "New legislation passed on climate change policy"
Code: politics
Reason: The article discusses government policy and legislation, which falls under political coverage.
"""

def prepare_prompt(text):
    system_prompt = f"""Instructions: You are an expert news classifier. Your task is to categorize BBC news articles into ONE of the following categories: business, entertainment, politics, sport, or tech.

Rules:
1. ALWAYS choose exactly ONE category
2. Base your decision on the main focus of the article
3. Format your response exactly as shown in the examples
4. Provide clear reasoning that references specific content from the article

Codebook:
{CODEBOOK}

Examples of correct classifications:
{EXAMPLES}

Now analyze this article:
---
{text}
---

Output your analysis in this exact format:
Code: [category]
Reason: [detailed explanation referencing specific content]"""
    return system_prompt

def classify_text(text, retry_count=3, retry_delay=1):
    """Classify text using local Ollama instance with optimized parameters."""
    for attempt in range(retry_count):
        try:
            # Generate response with optimized parameters
            response = ollama.generate(
                model='llama3.2:latest',
                prompt=prepare_prompt(text),
                stream=False,
                options={
                    'temperature': 0.1,  # Lower temperature for more consistent outputs
                    'top_p': 0.9,
                    'top_k': 40,
                    'num_predict': 200,  # Limit response length
                    'stop': ["\n\n", "Text:", "Example"]  # Stop tokens to prevent rambling
                }
            )
            
            output = response['response']
            
            if not output:
                raise ValueError("Empty response from model")
            
            print("\nRaw model output:", output)  # Debug output
                
            # More robust code extraction
            code_patterns = [
                r'Code:\s*(business|entertainment|politics|sport|tech)(?:\s|$)',  # Strict pattern
                r'Code:\s*([a-zA-Z]+)',  # Backup pattern
                r'\b(business|entertainment|politics|sport|tech)\b'  # Last resort
            ]
            
            code = None
            for pattern in code_patterns:
                match = re.search(pattern, output, re.IGNORECASE)
                if match:
                    code = match.group(1).lower().strip()
                    # Validate against allowed categories
                    if code in ['business', 'entertainment', 'politics', 'sport', 'tech']:
                        break
            
            # If no valid code found, use keyword matching
            if not code:
                text_lower = text.lower()
                keywords = {
                    'sport': ['football', 'soccer', 'tennis', 'rugby', 'player', 'match', 'game', 'tournament', 'championship'],
                    'business': ['market', 'economy', 'company', 'stock', 'trade', 'business', 'financial'],
                    'tech': ['technology', 'software', 'computer', 'digital', 'online', 'internet', 'app'],
                    'politics': ['government', 'minister', 'president', 'election', 'policy', 'parliament'],
                    'entertainment': ['movie', 'film', 'music', 'actor', 'star', 'celebrity', 'show']
                }
                
                # Count keyword matches for each category
                scores = {category: sum(1 for word in words if word in text_lower)
                         for category, words in keywords.items()}
                
                # Use category with most keyword matches if any found
                if max(scores.values()) > 0:
                    code = max(scores.items(), key=lambda x: x[1])[0]
                else:
                    code = "Unknown"
            
            # Extract reasoning
            reasoning_pattern = r'Reason:\s*(.*?)(?=(?:Code:|$))'
            reasoning_match = re.search(reasoning_pattern, output, re.IGNORECASE | re.DOTALL)
            reasoning = reasoning_match.group(1).strip() if reasoning_match else output.strip()
            
            # Validate and clean reasoning
            if len(reasoning) < 10:  # If reasoning too short, use raw output
                reasoning = output.strip()
            
            return code, reasoning
                
        except Exception as e:
            if attempt == retry_count - 1:
                return "Error", f"Processing failed after {retry_count} attempts: {str(e)}"
            time.sleep(retry_delay * (attempt + 1))
            continue

def process_dataframe(df, batch_size=50, save_interval=200):
    """Process the dataframe with progress tracking and periodic saving."""
    df_processed = df.copy()
    
    # Initialize columns for results
    if 'model_prediction' not in df_processed.columns:
        df_processed['model_prediction'] = None
    if 'reasoning' not in df_processed.columns:
        df_processed['reasoning'] = None
    
    unprocessed_mask = df_processed['model_prediction'].isna()
    total_unprocessed = unprocessed_mask.sum()
    
    if total_unprocessed == 0:
        print("All rows have already been processed!")
        return df_processed
    
    print(f"Processing {total_unprocessed} unprocessed rows...")
    
    with tqdm(total=total_unprocessed) as pbar:
        processed_count = 0
        
        for idx in df_processed[unprocessed_mask].index:
            try:
                text = df_processed.loc[idx, 'text']
                code, reasoning = classify_text(text)
                
                df_processed.loc[idx, 'model_prediction'] = code
                df_processed.loc[idx, 'reasoning'] = reasoning
                
                processed_count += 1
                pbar.update(1)
                
                # Save intermediate results
                if processed_count % save_interval == 0:
                    print(f"\nSaving intermediate results after processing {processed_count} rows...")
                    df_processed.to_csv('ollama_intermediate_results.csv', index=False)
                
                # Add delay to prevent overwhelming the model
                if processed_count % batch_size == 0:
                    time.sleep(1)
                    
            except Exception as e:
                print(f"\nError processing row {idx}: {str(e)}")
                continue
    
    # Print summary
    success_count = df_processed['model_prediction'].notna().sum() - df_processed['model_prediction'].eq('Error').sum()
    error_count = df_processed['model_prediction'].eq('Error').sum()
    print(f"\nProcessing complete!")
    print(f"Successfully processed: {success_count} rows")
    print(f"Errors: {error_count} rows")
    
    return df_processed

# Example usage:
if __name__ == "__main__":
    df_results2 = process_dataframe(df)
    print("Ready to process! Load your DataFrame and call process_dataframe()")

Processing 98 unprocessed rows...


  1%|          | 1/98 [00:10<16:31, 10.22s/it]


Raw model output: Code: politics
Reason: The article discusses a contract dispute between Bath rugby player Mike Tindall and his current club, with Tindall's agent warning that the club must improve their offer or risk losing him to another team. This falls under political coverage as it involves negotiations between a sports organization (Bath) and an individual athlete (Tindall), which is subject to the influence of external factors such as rival clubs and the club's owner, Andrew Brownsword.


  2%|▏         | 2/98 [00:14<11:03,  6.91s/it]


Raw model output: Code: sport
Reason: This article is primarily focused on tennis matches, featuring the progress of top-seeded players Andy Roddick and Andre Agassi through the SAP Open tournament. The text describes match outcomes, player performances, and quotes from the athletes themselves, which are all characteristic of sports reporting.


...


Raw model output: Code: tech
Reason: The article discusses the blocking of a news site, specifically Google News, and its implications on internet access and censorship. It also mentions Google's response to the issue and the technical aspects of how the news service is curated, such as the use of computer algorithms to select headlines. Additionally, it highlights China's extensive control over the internet, including monitoring websites and e-mails, which suggests a focus on technological and internet-related issues.


 99%|█████████▉| 97/98 [10:13<00:05,  5.71s/it]


Raw model output: Code: sport
Reason: This article is primarily focused on a rugby match between Leeds and Saracens, including team selections, player movements, and game details. The language used, such as "Tykes", "prop", "fly-half", and "pack", also suggests a sports context.


100%|██████████| 98/98 [10:19<00:00,  6.32s/it]


Raw model output: Code: tech
Reason: The article discusses biometric technology and its implementation on a luxury cruise liner, specifically mentioning prototype versions of internationally issued biometric ID cards and machine readers produced by different companies. This focus on technological advancements and innovation in the field of biometrics classifies the article as tech news.

Processing complete!
Successfully processed: 98 rows
Errors: 0 rows
Ready to process! Load your DataFrame and call process_dataframe()


In [14]:
results = df_results2

In [15]:
def calculate_similarity_metrics(results_df, ground_truth_col, prediction_col):
    """
    Calculate various similarity metrics between two columns
    """
    # Remove rows where either column has missing values
    valid_rows = results_df[[ground_truth_col, prediction_col]].dropna()
    
    if len(valid_rows) == 0:
        return {
            'accuracy': 0,
            'kappa': 0,
            'matching_cases': 0,
            'total_cases': 0,
            'percentage_match': 0
        }
    
    # Calculate metrics
    accuracy = accuracy_score(valid_rows[ground_truth_col], valid_rows[prediction_col])
    kappa = cohen_kappa_score(valid_rows[ground_truth_col], valid_rows[prediction_col])
    matching_cases = (valid_rows[ground_truth_col] == valid_rows[prediction_col]).sum()
    total_cases = len(valid_rows)
    percentage_match = (matching_cases / total_cases) * 100
    
    return {
        'accuracy': accuracy,
        'kappa': kappa,
        'matching_cases': matching_cases,
        'total_cases': total_cases,
        'percentage_match': percentage_match
    }

def analyze_code_similarities(results_df):
    """
    Analyze similarities between model outputs and human categorizations
    """
    # Comparisons to make
    comparisons = [
        ('original_code', 'model_code'),
        ('original_code', 'model_prediction'),
        ('replicated_code', 'model_code'),
        ('replicated_code', 'model_prediction')
    ]
    
    # Calculate metrics for each comparison
    results_dict = {}
    for truth_col, pred_col in comparisons:
        results_dict[f"{truth_col}_vs_{pred_col}"] = calculate_similarity_metrics(
            results_df, truth_col, pred_col
        )
    
    # Create a summary dataframe
    summary_df = pd.DataFrame({
        'Comparison': [k.replace('_', ' ').title() for k in results_dict.keys()],
        'Accuracy': [v['accuracy'] for v in results_dict.values()],
        'Kappa Score': [v['kappa'] for v in results_dict.values()],
        'Matching Cases': [v['matching_cases'] for v in results_dict.values()],
        'Total Cases': [v['total_cases'] for v in results_dict.values()],
        'Match Percentage': [v['percentage_match'] for v in results_dict.values()]
    })
    
    # Format the percentage column
    summary_df['Match Percentage'] = summary_df['Match Percentage'].round(2).astype(str) + '%'
    
    # Create detailed confusion matrices
    confusion_matrices = {}
    for truth_col, pred_col in comparisons:
        valid_rows = results_df[[truth_col, pred_col]].dropna()
        if len(valid_rows) > 0:
            matrix = confusion_matrix(
                valid_rows[truth_col], 
                valid_rows[pred_col],
                labels=sorted(results_df[truth_col].unique())
            )
            confusion_matrices[f"{truth_col}_vs_{pred_col}"] = matrix
    
    return summary_df, confusion_matrices

# Run the analysis with the results dataframe
summary_df, confusion_matrices = analyze_code_similarities(results)

# Display the summary
print("\nSimilarity Analysis Summary:")
print("=" * 100)
print(summary_df.to_string(index=False))
print("\nDetailed Analysis:")
print("=" * 100)

# Determine which model is more similar to human coding
original_code_comparison = summary_df[summary_df['Comparison'].str.contains('Original Code')]
replicated_code_comparison = summary_df[summary_df['Comparison'].str.contains('Replicated Code')]

print("\nComparison with Original Code:")
better_for_original = original_code_comparison.iloc[original_code_comparison['Accuracy'].argmax()]
print(f"Better model: {better_for_original['Comparison']}")
print(f"Accuracy: {better_for_original['Accuracy']:.2%}")
print(f"Kappa Score: {better_for_original['Kappa Score']:.3f}")

print("\nComparison with Replicated Code:")
better_for_replicated = replicated_code_comparison.iloc[replicated_code_comparison['Accuracy'].argmax()]
print(f"Better model: {better_for_replicated['Comparison']}")
print(f"Accuracy: {better_for_replicated['Accuracy']:.2%}")
print(f"Kappa Score: {better_for_replicated['Kappa Score']:.3f}")

# Print confusion matrices with labels
for name, matrix in confusion_matrices.items():
    print(f"\nConfusion Matrix for {name}:")
    categories = sorted(results[name.split('_vs_')[0]].unique())
    print("\nCategories:", categories)
    print(matrix)


Similarity Analysis Summary:
                         Comparison  Accuracy  Kappa Score  Matching Cases  Total Cases Match Percentage
        Original Code Vs Model Code  0.877551     0.842823              86           98           87.76%
  Original Code Vs Model Prediction  0.785714     0.725454              77           98           78.57%
      Replicated Code Vs Model Code  0.816327     0.761557              80           98           81.63%
Replicated Code Vs Model Prediction  0.704082     0.616153              69           98           70.41%

Detailed Analysis:

Comparison with Original Code:
Better model: Original Code Vs Model Code
Accuracy: 87.76%
Kappa Score: 0.843

Comparison with Replicated Code:
Better model: Replicated Code Vs Model Code
Accuracy: 81.63%
Kappa Score: 0.762

Confusion Matrix for original_code_vs_model_code:

Categories: ['business', 'entertainment', 'politics', 'sport', 'tech']
[[15  0  5  0  0]
 [ 2 14  1  0  0]
 [ 0  0 25  0  0]
 [ 0  0  2 23  0]
 [ 1  

In [16]:
df_results.to_csv('results_csvs/bbc_new_small_llama_first.csv', index=False)
df_results2.to_csv('results_csvs/bbc_new_small_llama_final.csv', index=False)